In [144]:
# package imports
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from pathlib import Path

import re
import pandas as pd
import logging
import traceback

# from joblib import Memory
from collections import deque
import time
from datetime import datetime
import mimetypes
from pprint import pprint


In [190]:
# @memory.cache
# def fetch_webpage(url, headers=HEADERS):
#     try:
#         response = requests.get(url, headers=headers)
#         return response
#     except Exception as e:
#         logger.error(e)
#         return None

def fetch_webpage(url, headers=HEADERS):
    response = requests.get(url, headers=headers)
    response.raise_for_status() # raise exception if response was not successful
    return response
    
# def get_soup(url_append, root_url=root_url, headers=HEADERS):
#     url = root_url + url_append
#     # TODO: implement caching using joblib
#     # cached_result = cache.get('html:%s' % url)
#     # if cached_result:
#     #     return cached_result
#     print(f"Getting BeautifulSoup object from {url}")
#     try:
#         response = requests.get(url, headers=headers)
#         # cache.set('html:%s' % url, page)
#         bs = BeautifulSoup(response.content,'html.parser')
#         return bs
#     except Exception as e:
#         print("Error: ", e)
#         return None
    
# def get_item(bs:BeautifulSoup, items, parent_path="/", is_folder=True):
#     ''' 
#     parent_path (str):
#         The path to the current folder, relative to the root of the website. Must start with a slash. At least one slash is required to indicate the root of the website.
#     '''
#     if is_folder:
#         class_str = 'folder-link'
#     else:
#         class_str = 'file-link'

#     for item in bs.find_all("a", class_=class_str):
#         if item.get('href') not in [i["url"] for i in items]:
#             item_info  = {
#                 "name": item.text, 
#                 "url": item.get('href'), 
#                 "parent_path": parent_path
#             }
#             items.append(item_info)
#     return items
    
def get_folders(bs:BeautifulSoup, folders:deque, parent_url:str, parent):
    ''' 
    parents : list(str)
        List of the folders on the path to the current folder, relative to the root of the website. parent_path[0] should be the first folder on the path, and parent_path[-1] is the current folder. parent_path=[] indicates that the folder belongs to the root.
    '''
    logger.debug("folder parents: %s", parent)
    for folder in bs.find_all("a", class_='folder-link'):
        folder_info = {
                "name": folder.text.strip(), 
                "url": folder.get('href'),
                "parent": parent,
                "parent_url": parent_url
            }
        logger.debug(f"{folder}")

        if folder.get('href') not in [f["url"] for f in folders]:
            logger.debug("Adding to folders deque to visit")
            folders.append(folder_info)
    return folders

def get_items(bs:BeautifulSoup, parent_url:str, parent=[], is_folder=True)->list[dict]:
    ''' 
    Return a list of dictionaries containing information for each item in the current folder. Items are either documents or folders.
    
    Parameters
    ----------
    bs: BeautifulSoup
        The BeautifulSoup object to parse. Should be the documents site for a website under the civicweb.net domain.
    parent_url : str
        The url of the current folder. This is the url appended onto the root of the website, and should start with "/".
    parent : list(str)
        List of the folders on the path to the current item, relative to the root of the website. parent[0] should be the first folder on the path, and parent[-1] is the current folder. parent=[] indicates that the folder belongs to the root.
    is_folder  : bool
        Whether or not the current folder is a folder (True) or a document (False).
    
    Returns
    -------
    items : list[dict]
        A list of dictionaries containing information for each item in the current folder. Each dictionary contains the following keys:
        - name : The name of the item.
        - url : The url of the item.
        - parent_url : The url of the parent folder.
        - parent : A list of the folders on the path to the current item.
    '''
    logger.debug("folder parent: %s", parent)

    child_items = []
    if is_folder: 
        item_class = "folder-link" 
    else: 
        item_class = "document-link" 

    for item in bs.find_all("a", class_=item_class):
        item_info = {
                "name": item.text.strip(), 
                "url": item.get('href'),
                "parent": parent,
                "parent_url": parent_url
            }
        logger.debug(f"Found {item_class} with name {item_info['name']} (url: {item_info['url']}) in directory at {'/'.join(item_info['parent'])} (url: {parent_url}).")
        child_items.append(item_info)
    return child_items

def get_documents(bs:BeautifulSoup, documents:list, parent_url:str, parent:list):
    for doc in bs.find_all("a", class_='document-link'):
        document_info = {
                "name": doc.text.strip(),
                "url": doc["href"],
                "parent": parent,
                "parent_url": parent_url
            }
        logger.debug(f"Found document with name {document_info['name']} in {'/'.join(document_info['parent'])}, fetched from {document_info['url']}")
        if doc.get('href') not in [d["url"] for d in documents]:
            logger.debug(f"Adding {document_info['name']} to documents list to download later")
            documents.append(document_info)
    return documents

def get_filetype(response:requests) -> tuple[str, str]:
    ''' 
    Returns the extension and file type (MIME notation) of a response object based on its Content-Type header.
    '''
    header_mimetype = response.headers['Content-Type']

    # remove mimetype parameter if it exists
    if ";" in header_mimetype:
        header_mimetype = header_mimetype.split(";")[0]
    extension = mimetypes.guess_extension(header_mimetype)

    # raise error if file extension cannot be guessed from response
    if extension is None or extension == "":
        logger.warning("Could not determine file type for response")
        raise TypeError(f"Could not determine file type")
    
    return extension, header_mimetype

# def format_output_path(parents:list, subdomain="", out_folder=Path.cwd() / "out"):
#     '''
#     Formats the output path of the file to be downloaded. The output path will separate the file by subdomain and preserve the file structure from the scraped website.

#     parents : list(str)
#         The list of directories that the file is in from the root folder of the main website.
#     subdomain : str, optional
#         The subdomain of the website to be scraped from. Defaults to "". Used to separate scraped content into specific folders within the out_path.
#     '''
#     return out_folder.joinpath(subdomain, *parents)

def download_file(response:requests, filename:str,  out_path=Path.cwd() / "out"):
    ''' 
    Downloads a PDF file from the given response.
    
    Parameters
    ----------
    response : requests.Response
        The response object.
    filename : str
        The name of the file being downloaded. Should include the extension.
    
    out_path : Path, optional
        The path where the file should be saved on the disk, as a pathlib Path object. Defaults to a folder in the current working directory named "out". Does not need to exist.
    '''
    # out_path = Path(__file__).parent / out_path / subdomain / Path(*parents) / name + ".pdf" # use Path.cwd() for Jupyter Notebook

    out_path.mkdir(parents=True, exist_ok=True) # ensure directories leading up to the output file path exist
    with open(out_path / filename, "wb") as f:
        f.write(response.content)
        f.close()

In [119]:
logging.basicConfig(format='%(asctime)s - %(levelname)s:%(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# logger.setLevel(logging.INFO)

In [120]:
import requests_cache

requests_cache.install_cache('scraper_cache', backend='sqlite', expire_after=3600*24*14)

In [130]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}

In [ ]:
# requests_cache.clear() 

In [ ]:
# cachedir = './cache' 
# memory = Memory(cachedir, verbose=0)

# finding all the subdomains


In [ ]:
# 'https://winchesterva.civicweb.net'

#### using bing search API

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
API_KEY = ""
api_header = {
    "Ocp-Anim-Subscription-Key": API_KEY,
}

In [ ]:
bing_api_url = 'https://api.bing.microsoft.com/v7.0/search'

In [ ]:
api_query = {
    "q": "site:civicweb.net", # or 'site%3acivicweb.net'
    "mkt": "en-CA", # for results from Canada; referenced from https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/reference/market-codes
    # "count": 50,
    # "offset": 0,
}

#### using selenium

In [160]:
HEADERS1={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [159]:
scraped_links = []

In [176]:
driver = webdriver.Firefox()

for i in range(10):
    # time.sleep(1)
    page_number = 1+i*10 # default number of searches per bing page is 10
    bing_url = f'https://www.bing.com/search?q=site%3acivicweb.net&first={page_number}' # search with query

    driver.get(bing_url)
    links = driver.find_elements(by=By.TAG_NAME, value="cite")

    for link in links:
        scraped_links.append(link.text)

In [177]:
scraped_links

['https://victoria.civicweb.net/Portal/Welcome.aspx',
 'governmentjobs.com',
 'tender.victoria.ca',
 'melbourneflorida.org',
 'victoriatx.gov',
 'ci.victoria.mn.us',
 'https://victoria.civicweb.net/Portal',
 'victoria.civicweb.net',
 'victoria.civicweb.net',
 'https://tay.civicweb.net/Portal',
 'https://terrace.civicweb.net/Portal/Default.aspx',
 'https://cityofshawnee.civicweb.net/portal',
 'https://loyalist.civicweb.net/portal',
 'https://victoria.civicweb.net/user/signin',
 'https://wetaskiwin.civicweb.net/Portal/Default.aspx',
 'https://mclendon-chisholm.civicweb.net/Portal',
 'https://revelstoke.civicweb.net/portal',
 'https://victoria.civicweb.net/Portal',
 'https://victoria.civicweb.net/Portal/Welcome.aspx',
 'https://kamloops.civicweb.net/Portal/Default.aspx',
 'https://cityofjerseycity.civicweb.net/Portal',
 'https://wifn.civicweb.net/portal',
 'https://nngov.civicweb.net/portal/members.aspx?id=10',
 'https://centralelgin.civicweb.net',
 'https://revelstoke.civicweb.net/portal

In [179]:
subdomain_list = []

In [180]:
for link in scraped_links:
    if "civicweb.net" in link:
        logger.debug(f"found site {link}")
        subdomain = link.split(".civicweb.net")[0].split("https://")[-1]
        if subdomain not in subdomain_list:
            subdomain_list.append(subdomain)

2024-07-27 15:57:08 - DEBUG:found site https://victoria.civicweb.net/Portal/Welcome.aspx
2024-07-27 15:57:08 - DEBUG:found site https://victoria.civicweb.net/Portal
2024-07-27 15:57:08 - DEBUG:found site victoria.civicweb.net
2024-07-27 15:57:08 - DEBUG:found site victoria.civicweb.net
2024-07-27 15:57:08 - DEBUG:found site https://tay.civicweb.net/Portal
2024-07-27 15:57:08 - DEBUG:found site https://terrace.civicweb.net/Portal/Default.aspx
2024-07-27 15:57:08 - DEBUG:found site https://cityofshawnee.civicweb.net/portal
2024-07-27 15:57:08 - DEBUG:found site https://loyalist.civicweb.net/portal
2024-07-27 15:57:08 - DEBUG:found site https://victoria.civicweb.net/user/signin
2024-07-27 15:57:08 - DEBUG:found site https://wetaskiwin.civicweb.net/Portal/Default.aspx
2024-07-27 15:57:08 - DEBUG:found site https://mclendon-chisholm.civicweb.net/Portal
2024-07-27 15:57:08 - DEBUG:found site https://revelstoke.civicweb.net/portal
2024-07-27 15:57:08 - DEBUG:found site https://victoria.civicw

In [181]:
pprint(subdomain_list)
print(len(subdomain_list))

['victoria',
 'tay',
 'terrace',
 'cityofshawnee',
 'loyalist',
 'wetaskiwin',
 'mclendon-chisholm',
 'revelstoke',
 'kamloops',
 'cityofjerseycity',
 'wifn',
 'nngov',
 'centralelgin',
 'millcreek',
 'hartford',
 'otonabeesouthmonaghan',
 'greatermadawaska',
 'springwater',
 'westnewyorknj',
 'lewes',
 'opkansas',
 'orillia',
 'sequimwa',
 'lawrenceks',
 'powellriver',
 'sammamishwa',
 'peoriagov',
 'marmoraandlake',
 'cityofalice',
 'countygp',
 'selkirk',
 'southfrontenac',
 'cloquet',
 'pinecitygovoffice',
 'sphosp',
 'centrewellington',
 'dallascounty',
 'highriver',
 'citwentynine-palmsca',
 'osoyoos',
 'ramara',
 'petrolia',
 'timmins',
 'williamsnd',
 'lacenter',
 'codb',
 'woonsocketri',
 'covinaca',
 'stlouisco',
 'cityofholland',
 'mindenhills',
 'waverly',
 'englewoodgov',
 'winnipegsdca',
 'washingtoncounty',
 'hearst',
 'cityofbelmont',
 'severn',
 'lillooettribalcouncil',
 'stonemills',
 'muskoka',
 'cityofrehoboth',
 'walton',
 'peachland',
 'honeybrooktwp',
 'cocookmn'

In [197]:
subdomain_rows = [] # for the dataframe
# testing with error
subdomains_test = subdomain_list.copy()
subdomains_test.append("https://grangkrb.civicweb.net")
subdomains_with_documents = [] # valid site to be scraped
# for subdomain in subdomain_list:
for subdomain in subdomains_test:
    root_url = f"https://{subdomain}.civicweb.net"
    error = ""
    num_documents = 0
    # subdomain = root_url.split("https://")[-1].split(".civicweb.net")[0]
    # if subdomain returns code 200 and text is not empty, add to list of sites to scrape
    try:
        response = fetch_webpage(url=root_url+"/filepro/documents/") # get the url of the documents hub
        logger.debug(f"{root_url} has a documents page, adding to valid subdomains list")
        subdomains_with_documents.append(subdomain)
    except Exception as e:
        logger.error(e)
        error += str(e)
        continue
    finally:
        subdomain_info = {
            "subdomain": subdomain, 
            "root_url": root_url, 
            "date_scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "'error_in_accessing'": error,
        }
        subdomain_rows.append(subdomain_info)

2024-07-27 16:39:23 - DEBUG:https://victoria.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://tay.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://terrace.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://cityofshawnee.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://loyalist.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://wetaskiwin.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://mclendon-chisholm.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://revelstoke.civicweb.net has a documents page, adding to valid subdomains list
2024-07-27 16:39:23 - DEBUG:https://kamloops.civicweb.net has a documents page, adding to va

In [198]:
pd.DataFrame(subdomain_rows).to_csv(OUT_FOLDER / "scraped_subdomains_test.csv")

In [199]:
subdomains_df = pd.read_csv(OUT_FOLDER / "scraped_subdomains_test.csv", index_col=[0])
subdomains_df

,subdomain,root_url,date_scraped,'error'
0,victoria,https://victoria.civicweb.net,2024-07-27 16:39:23,NaN
1,tay,https://tay.civicweb.net,2024-07-27 16:39:23,NaN
2,terrace,https://terrace.civicweb.net,2024-07-27 16:39:23,NaN
3,cityofshawnee,https://cityofshawnee.civicweb.net,2024-07-27 16:39:23,NaN
4,loyalist,https://loyalist.civicweb.net,2024-07-27 16:39:23,NaN
...,...,...,...,...
74,delavan,https://delavan.civicweb.net,2024-07-27 16:39:23,NaN
75,indio,https://indio.civicweb.net,2024-07-27 16:39:23,NaN
76,camrose,https://camrose.civicweb.net,2024-07-27 16:39:23,NaN
77,bracebridge,https://bracebridge.civicweb.net,2024-07-27 16:39:23,NaN


In [ ]:
driver = webdriver.Firefox()

max_page_number = 10 # max number of pages to scrape
num_results_per_page = 10 # default number

for page_index in range(max_page_number):
    # time.sleep(1) # wait for 1 second between each search

    # search with query and result page
    start_index = 1+page_index*num_results_per_page
    logger.info(f"Scraping page {page_index+1} with results from {start_index} to {start_index+num_results_per_page-1}...")

    bing_url = f'https://www.bing.com/search?q=site%3acivicweb.net&first={start_index}' 

    # get url of each page result
    try:
        driver.get(bing_url)
        titles = driver.find_elements(by=By.CLASS_NAME, value="tptt")
        links = driver.find_elements(by=By.TAG_NAME, value="cite")
        snippets = driver.find_elements(by=By.CLASS_NAME, value="b_lineclamp4")

        bing_links.extend([{"bing_search_url":link, "title":title, "description":snippet} for (title, link, snippet) in zip(titles, links, snippets)])

    except WebDriverException as e:
        logger.error("Error opening Bing page for scraping: ", str(e))
        continue
    finally:
        driver.quit()
    
logger.info(f"Bing scraping finished. Found {len(bing_links)} links.")

In [ ]:
# subdomain_list = []

# for link in scraped_links:
#     if "civicweb.net" in link:
#         subdomain = link.split(".civicweb.net")[0].split("https://")[-1]
#         if subdomain not in subdomain_list:
#             logger.debug(f"Found new subdomain {subdomain}")
#             subdomain_list.append(subdomain)
#     else:
#         logger.debug(f"{link} is ambiguous")

# logger.info(f"Found {len(subdomain_list)} civicweb.net subdomains.")

In [182]:
# page_number = 1
# bing_url = f'https://www.bing.com/search?q=site%3acivicweb.net&first={page_number}'
# HEADERS1={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [183]:
# driver = webdriver.Firefox()

In [184]:
# driver.get(bing_url)

In [185]:
# links = driver.find_elements(by=By.TAG_NAME, value="cite")

In [186]:
# for link in links:
#     print(link.text)
#     scraped_links.append(link.text)

In [189]:
# scraped_links

In [187]:
# for link in scraped_links:
#     if "civicweb.net" in link:
#         print(f"found site {link}")
#         subdomain = link.split(".civicweb.net")[0].split("https://")[-1]
#         if subdomain not in subdomain_list:
#             subdomain_list.append(subdomain)

In [188]:
# subdomain_list

#### using requests

In [136]:
# response = requests.get(bing_url, headers=HEADERS1)
# response.text # returns nothing

''

In [126]:
bs = BeautifulSoup(response.text, 'html.parser')
bs.prettify()

''

## processing files on one website

In [ ]:
subdomain = 'pleasantontx'
root_url = f"https://{subdomain}.civicweb.net"
subdomain = root_url.split("https://")[-1].split(".civicweb.net")[0]

In [ ]:
folders = deque([]) 
done_folders = deque([])

In [ ]:
# testpath = Path("out")
# testpath = testpath.joinpath("", "test")
# testpath.mkdir(parents=True, exist_ok=True)

In [ ]:
# add the folders at the root to be processed
response = fetch_webpage(url=root_url+"/filepro/documents/")
bs = BeautifulSoup(response.content,'html.parser')
folders.extend(get_items(bs, parent_url="/filepro/documents/",parent=[], is_folder=True))

In [ ]:
folders

In [ ]:
documents = []

In [ ]:
documents

In [ ]:
# memory.clear()

In [ ]:
# breadth-first search to find all subfolders and documents
while len(folders)>0:
    time.sleep(1)
    logger.debug("folders to visit: %s", "\n".join([str(folder) for folder in folders]))
    logger.debug("completed folders: %s", "\n".join([str(folder) for folder in done_folders]))
    
    curr_folder = folders.popleft()
    logger.info(f"\nSearching folder {curr_folder['name']} at location /{'/'.join(curr_folder['parent'])}...") 
    try:
        response = fetch_webpage(url=root_url+curr_folder["url"])
        bs = BeautifulSoup(response.content,'html.parser')

        # update current folder
        curr_path = curr_folder["parent"].copy()
        curr_path.append(curr_folder["name"])
        logger.debug("currrent folder's parents:%s", curr_folder["parent"])
        logger.debug("current folder's name:%s", curr_folder["name"])
        logger.debug("parent path to add to children folders/documents: %s", curr_path)
        
        # add subfolders to visit from this folder to the folders deque
        # initial_folders = len(folders)
        children_folders = get_items(bs,parent_url=curr_folder["url"], parent=curr_path, is_folder=True)
        folders.extend(children_folders)

        # add documents to download from this folder to the documents list
        # initial_documents = len(documents)
        children_documents = get_items(bs,parent_url=curr_folder["url"], parent=curr_path, is_folder=False)
        documents.extend(children_documents)
        
        logger.info(f"Found {len(children_folders)} folders and {len(children_documents)} documents at this location.")
    except Exception as e:
        tb_str = ''.join(traceback.format_exception(e))
        logger.error(tb_str)
        continue

    done_folders.append(curr_folder)
    curr_path = []
    
    logger.info(f"Completed folder {curr_folder['name']}\n")

In [ ]:
print(folders)
print(done_folders)

In [ ]:
documents

## downloading documents

In [ ]:
# # https://pleasantontx.civicweb.net/filepro/documents/5688/ #has 4 documents
# bs = get_soup(url_append='/filepro/documents/5688/', root_url=root_url)

In [ ]:
# documents = []
# documents = get_documents(bs, documents)

In [ ]:
# documents

In [ ]:
# documents = []
# for doc in bs.find_all("a", class_='document-link'):
#     document_info = {
#         "name": doc.text.strip(),
#         "url": doc["href"],
#         "subdomain": subdomain,
#         }
#     documents.append(document_info)
# print(documents)

In [ ]:
len(documents)

In [ ]:
# def get_parent_url(document:dict, folders:deque=done_folders):
#     return [folder for folder in folders 
#     if "".join(folder["parents"])+folder["name"] == "".join(document["parents"])
#     ]

In [ ]:
OUT_FOLDER = Path.cwd() / "out"
rows = []
for document in documents[:10]:
    error = ""
    file_extension = ""
    try:
        t1 = time.time()
        response = fetch_webpage(url=root_url+document["url"], headers=HEADERS)
        t2 = time.time()
        logger.info((f"Took {round((t2-t1),3)} seconds to get page."))
        out_path = OUT_FOLDER.joinpath(subdomain, *document["parent"])

        file_extension, file_type = get_filetype(response)
        logger.info((f"{document['name']}{file_extension}"))
        download_file(
            response, 
            filename=document["name"]+file_extension,
            out_path=out_path)
    except Exception as e:
        tb_str = ''.join(traceback.format_exception(e))
        logger.error(tb_str)
        error += e # update with error
    finally:
        download_dict = {
            "name": document["name"]+file_extension,
            "file_type": file_type,
            "subdomain": subdomain,
            "parent_path": "/".join(document["parent"]),
            "root_url": root_url, 
            "url": document["url"], 
            "parent_url": document["parent_url"],
            "date_scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "'error_in_scraping'": error
        }
        rows.append(download_dict)

In [ ]:
rows

In [ ]:
out_df = pd.DataFrame(rows)
out_df

In [ ]:
out_df.to_csv(OUT_FOLDER / "scraped_files.csv")

In [ ]:
in_df = pd.read_csv(OUT_FOLDER / "scraped_files.csv", index_col=[0])

In [ ]:
in_df

In [ ]:
# t1 = time.time()
# response = requests.get(root_url+documents[0]["url"], headers=HEADERS)
# t2 = time.time()
# print(f"Took {round((t2-t1),3)} seconds to get page.")

# t3 = time.time()
# with open("./out/"+documents[0]["name"]+".pdf", 'wb') as pdf:
#     pdf.write(response.content)
#     pdf.close()
# t4 = time.time()
# print(f"Took {round((t4-t3),3)} seconds to save pdf.")

In [ ]:
# documents[0] and [1] are in folder /filepro/documents/5869 
response = fetch_webpage(url_append='/filepro/documents/5869', root_url=root_url, headers=HEADERS)
bs = BeautifulSoup(response.content,'html.parser')

In [ ]:
response = fetch_webpage(url_append=documents[0]["url"], root_url=root_url, headers=HEADERS)
bs = BeautifulSoup(response.content)

In [ ]:
download_file(response, filename=documents[0]["name"]+".html", parents=documents[0]["parents"], subdomain=subdomain, out_path = Path.cwd() / "out")

In [ ]:
response.headers['Content-Type']

In [ ]:
response.headers['Content-Type'].split(";")[0]

In [ ]:
# print(mimetypes.guess_extension(response.headers['Content-Type'])) # results in None
print(mimetypes.guess_extension(response.headers['Content-Type'].split(";")[0]))

In [ ]:
print(mimetypes.guess_extension('text/html'))

In [ ]:
print(mimetypes.guess_extension(response1.headers['Content-Type']))

In [ ]:
mimetypes.types_map['.html']

In [ ]:
print(response.headers)

In [ ]:
response1 = fetch_webpage(url_append=documents[1]["url"], root_url=root_url, headers=HEADERS)
bs1 = BeautifulSoup(response1.content)

In [ ]:
print(response1.headers)

In [ ]:
print(response1.headers["Content-Disposition"])

In [ ]:
print(response1.headers.keys)

In [ ]:
print(response.headers["Content-Type"])

In [ ]:
response = fetch_webpage(url_append=documents[1]["url"], root_url=root_url, headers=HEADERS)
bs = BeautifulSoup(response.content)

In [ ]:
print(response.headers["Content-Type"])

In [ ]:
mimetypes.guess_extension('image/jpeg')

# Bing search

In [ ]:
# BING_SEARCH_ENDPOINT = 'https://api.bing.microsoft.com/v7.0/search'

# API_KEY = os.getenv('BING_API') # Need to obtain key from https://portal.azure.com/. Create a ".env" file in the root folder and add the key to an API_KEY variable inside the ".env" file.
# api_header = {
#     "Ocp-Anim-Subscription-Key": API_KEY,  
# }
# api_params = {
#     # "q": "site:civicweb.net", # or 
#     "q": 'site%3acivicweb.net',
#     "mkt": "en-CA", # for results from Canada; referenced from https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/reference/market-codes
#     # "count": 50,
#     # "offset": 0,
# }

# response = session.get(BING_SEARCH_ENDPOINT, headers=api_header, params=api_params)
# response.raise_for_status()
# search_results = response.json()

## selenium scraping